In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

In [2]:
df = pd.read_csv('C:/Users/whgod/Desktop/머신러닝/PHW-Lab1/Lab1/breast-cancer-wisconsin.data',header=None)
df.columns = ['ID', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion',
             'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']

In [3]:
df.head()

,ID,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           699 non-null    int64 
 1   Clump Thickness              699 non-null    int64 
 2   Uniformity of Cell Size      699 non-null    int64 
 3   Uniformity of Cell Shape     699 non-null    int64 
 4   Marginal Adhesion            699 non-null    int64 
 5   Single Epithelial Cell Size  699 non-null    int64 
 6   Bare Nuclei                  699 non-null    object
 7   Bland Chromatin              699 non-null    int64 
 8   Normal Nucleoli              699 non-null    int64 
 9   Mitoses                      699 non-null    int64 
 10  Class                        699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [5]:
df.describe()

,ID,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [6]:
print(df['Bare Nuclei'].unique())

['1' '10' '2' '4' '3' '9' '7' '?' '5' '8' '6']


In [7]:
df.replace('?',np.nan, inplace=True)

df.dropna(subset=['Bare Nuclei'], axis=0, inplace=True)
df['Bare Nuclei'] = df['Bare Nuclei'].astype(int)
df.describe()

,ID,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   ID                           683 non-null    int64
 1   Clump Thickness              683 non-null    int64
 2   Uniformity of Cell Size      683 non-null    int64
 3   Uniformity of Cell Shape     683 non-null    int64
 4   Marginal Adhesion            683 non-null    int64
 5   Single Epithelial Cell Size  683 non-null    int64
 6   Bare Nuclei                  683 non-null    int32
 7   Bland Chromatin              683 non-null    int64
 8   Normal Nucleoli              683 non-null    int64
 9   Mitoses                      683 non-null    int64
 10  Class                        683 non-null    int64
dtypes: int32(1), int64(10)
memory usage: 61.4 KB


In [9]:
X = df.drop(columns = ['ID','Class'])
y = df['Class']

In [10]:
scaler_s = StandardScaler()
scaled_s = scaler_s.fit_transform(X) # array 형태
scaled_s = pd.DataFrame(scaled_s)
scaled_s

,0,1,2,3,4,5,6,7,8
0,0.197905,-0.702212,-0.741774,-0.639366,-0.555608,-0.698853,-0.181827,-0.612927,-0.348400
1,0.197905,0.277252,0.262783,0.758032,1.695166,1.772867,-0.181827,-0.285105,-0.348400
2,-0.511643,-0.702212,-0.741774,-0.639366,-0.555608,-0.424217,-0.181827,-0.612927,-0.348400
3,0.552679,1.583204,1.602192,-0.639366,-0.105454,0.125054,-0.181827,1.354008,-0.348400
4,-0.156869,-0.702212,-0.741774,0.059333,-0.555608,-0.698853,-0.181827,-0.612927,-0.348400
...,...,...,...,...,...,...,...,...,...
678,-0.511643,-0.702212,-0.741774,-0.639366,-0.105454,-0.424217,-0.998853,-0.612927,-0.348400
679,-0.866417,-0.702212,-0.741774,-0.639366,-0.555608,-0.698853,-0.998853,-0.612927,-0.348400
680,0.197905,2.236180,2.271896,0.059333,1.695166,-0.149582,1.860738,2.337476,0.229166
681,-0.156869,1.583204,0.932487,0.408682,-0.105454,0.125054,2.677764,1.026185,-0.348400


In [11]:
def AutoScale(DataFrame, scalers) :
    
    """
    It receives the desired data frame and scale function as variables and returns a data frame.
    --------------------------------
    DataFrame : DataFrame to scale
    scalers : The scaler you want to apply
    """
    scaler = scalers()
    scaled = scaler_s.fit_transform(DataFrame)
    scaled = pd.DataFrame(scaled)
    
    return scaled

In [12]:
def Compare_accuracy(predictor, target, estimators, param_grids, n_splits):
    
    """
    Compare the performance (accuracy) of different classification models on the same data set.
    print model, best parameters and best score.
    --------------------------------
    predictor : DataFrame of predictors
    target : DataFrame of target variables
    estimators : the model you want to compare
            None : [DecisionTreeClassifier(criterion='gini'), DecisionTreeClassifier(criterion='entropy'), 
                   LogisticRegression(), SVC()]
    param_grids : Parameters you want to apply for each model
            None : [ { "max_depth" : range(1,10), "max_features" : ['auto', 'sqrt', 'log2'], 
                        "min_samples_split" : range(2,10), "min_samples_leaf" : range(1,5) },
                     { "max_depth" : range(1,10), "max_features" : ['auto', 'sqrt', 'log2'],
                         "min_samples_split" : range(2,10), "min_samples_leaf" : range(1,5) },
                    { "penalty" : ['none', 'l2'], "C" : [0.1, 1, 3, 5] },
                    { "kernel" : ['linear', 'rbf'], "C" : [ 0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 100 ], 
                        'gamma' : [ 'scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100 ] } ]
    n_splits : numbers k for k-fold cross validation
            None : 10
    """   
    X_train, X_test, y_train, y_test = train_test_split(predictor, target,
                                                        test_size = 0.2, random_state = 42)
    sm = SMOTE(random_state = 42)
    X_res, y_res = sm.fit_resample(X_train, y_train)
        
    if estimators == None:
        estimator = [DecisionTreeClassifier(criterion='gini'),
                     DecisionTreeClassifier(criterion='entropy'),
                     LogisticRegression(),
                     SVC()]
    else: estimator = estimators
    
    if param_grids == None:
        param_grid = [
            { "max_depth" : range(1,10), "max_features" : ['auto', 'sqrt', 'log2'], 
             "min_samples_split" : range(2,10), "min_samples_leaf" : range(1,5) },
            { "max_depth" : range(1,10), "max_features" : ['auto', 'sqrt', 'log2'],
             "min_samples_split" : range(2,10), "min_samples_leaf" : range(1,5) },
            { "penalty" : ['l2','none'], "C" : [0.1, 1, 3, 5]},
            { "kernel" : ['linear', 'rbf'], "C" : [ 0.001, 0.01, 0.1, 1, 3, 5, 7, 10 , 100 ], 
              'gamma' : ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100 ] }
        ]
    else: param_grid = param_grids
    
    if n_splits == None:
        n_split = 10
    else : n_split = n_splits
    
    for i,j in zip(estimator, param_grid):
        print('-------','model:', str(i), '-------')
        grid_search = GridSearchCV(estimator = i,
                                   param_grid = j,
                                   cv = n_split,
                                   n_jobs = -1,
                                   verbose = 1)
        grid_search.fit(X_res, y_res)
        print('best parameters :' , grid_search.best_params_)
        print('best score :', grid_search.best_score_)
        em = grid_search.best_estimator_
        pred = em.predict(X_test)
        print('accuracy :', accuracy_score(y_test, pred), '\n')

In [13]:
Compare_accuracy(X, y, None, None, 3)

------- model: DecisionTreeClassifier() -------
Fitting 3 folds for each of 864 candidates, totalling 2592 fits
best parameters : {'max_depth': 7, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 7}
best score : 0.971238390789089
accuracy : 0.9708029197080292 

------- model: DecisionTreeClassifier(criterion='entropy') -------
Fitting 3 folds for each of 864 candidates, totalling 2592 fits
best parameters : {'max_depth': 9, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 5}
best score : 0.9726157547954756
accuracy : 0.948905109489051 

------- model: LogisticRegression() -------
Fitting 3 folds for each of 8 candidates, totalling 24 fits
best parameters : {'C': 0.1, 'penalty': 'l2'}
best score : 0.9725876453259574
accuracy : 0.9708029197080292 

------- model: SVC() -------
Fitting 3 folds for each of 144 candidates, totalling 432 fits
best parameters : {'C': 7, 'gamma': 0.001, 'kernel': 'rbf'}
best score : 0.9753255076570194
accuracy : 0.98540145